<a href="https://colab.research.google.com/github/gh-ita/StudentProductivity/blob/main/ProblemSolving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
dataset_path = '/content/student_data.csv'
student_df = pd.read_csv(dataset_path)